<a href="https://colab.research.google.com/github/sudeepmenon17/PytorchforNN/blob/main/pytorch_to_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-auth
from google.colab import auth
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
  # !pip install torchvision
!pip install onnx
  # !pip install tensorflow
!pip install onnx_tf
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf

In [ ]:
# # Define the Model class
# class Model(nn.Module):
#     def __init__(self, in_features=5, h1=61, h2=60, out_features=1):
#         super(Model, self).__init__()
#         self.fc1 = nn.Linear(in_features, h1)
#         self.fc2 = nn.Linear(h1, h2)
#         self.out = nn.Linear(h2, out_features)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         x = self.sigmoid(self.fc1(x))
#         x = self.sigmoid(self.fc2(x))
#         x = self.out(x)
#         return x

In [ ]:
# # Step 1: Export PyTorch model to ONNX format
# def export_pytorch_to_onnx(model, dummy_input, onnx_file_path):
#     torch.onnx.export(
#         model,
#         dummy_input,
#         onnx_file_path,
#         export_params=True,
#         opset_version=11,
#         do_constant_folding=True,
#         input_names=['input'],
#         output_names=['output']
#     )

# # Step 2: Convert ONNX model to TensorFlow SavedModel format
# def convert_onnx_to_tf(onnx_file_path, tf_export_path):
#     onnx_model = onnx.load(onnx_file_path)
#     tf_rep = prepare(onnx_model)
#     tf_rep.export_graph(tf_export_path)

# # Load the saved PyTorch model
# model = Model()
# model.load_state_dict(torch.load('/content/drive/MyDrive/airfoil_model.pt'))
# model.eval()

# # Create a dummy input tensor with the same shape as your input data
# dummy_input = Variable(torch.randn(1, 5))  # Adjust shape as necessary

# # Define file paths
# onnx_file_path = "/content/drive/MyDrive/airfoil_model.onnx"
# tf_export_path = "/content/drive/MyDrive/airfoil_model_tf"

# # Perform the conversion
# export_pytorch_to_onnx(model, dummy_input, onnx_file_path)
# convert_onnx_to_tf(onnx_file_path, tf_export_path)

# print(f"Model successfully converted to TensorFlow SavedModel format and saved in {tf_export_path} directory")


In [ ]:

import torch
import torch.onnx
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf

# Define the PyTorch model class
class Model(nn.Module):
    def __init__(self, in_features=6, h1=61, h2=60, out_features=1):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.out(x)
        return x

# Step 1: Export PyTorch model to ONNX format
def export_pytorch_to_onnx(model, dummy_input, onnx_file_path):
    torch.onnx.export(
        model,
        dummy_input,
        onnx_file_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output']
    )

# Step 2: Convert ONNX model to TensorFlow SavedModel format
def convert_onnx_to_tf(onnx_file_path, tf_export_path):
    onnx_model = onnx.load(onnx_file_path)
    tf_rep = prepare(onnx_model)
    tf_rep.export_graph(tf_export_path)

# Step 3: Load the TensorFlow model and re-save it using tf.keras.models.save_model()
def resave_tf_model(tf_export_path, final_tf_export_path):
    class KerasModel(tf.keras.Model):
        def __init__(self):
            super(KerasModel, self).__init__()
            self.fc1 = tf.keras.layers.Dense(61, activation='sigmoid')
            self.fc2 = tf.keras.layers.Dense(60, activation='sigmoid')
            self.out = tf.keras.layers.Dense(1)

        def call(self, x):
            x = self.fc1(x)
            x = self.fc2(x)
            x = self.out(x)
            return x

    # Load the TensorFlow SavedModel
    model = tf.saved_model.load(tf_export_path)

    # Create an instance of the Keras model
    keras_model = KerasModel()

    # Build the Keras model with the input shape
    keras_model.build(input_shape=(None, 6))

    # Perform a forward pass with dummy data to ensure initialization
    dummy_input = tf.random.normal([1, 6])
    keras_model(dummy_input)

    # Save the Keras model
    tf.keras.models.save_model(keras_model, final_tf_export_path)

# Load the saved PyTorch model
model = Model()
model.load_state_dict(torch.load('/content/drive/MyDrive/airfoil_model.pt'))
model.eval()

# Create a dummy input tensor with the same shape as your input data
dummy_input = Variable(torch.randn(1, 6))  # Adjust shape as necessary

# Define file paths
onnx_file_path = "/content/drive/MyDrive/airfoil_model.onnx"
tf_export_path = "/content/drive/MyDrive/airfoil_model_tf"
final_tf_export_path = "/content/drive/MyDrive/airfoil_model_tf_final"

# Perform the conversion
export_pytorch_to_onnx(model, dummy_input, onnx_file_path)
convert_onnx_to_tf(onnx_file_path, tf_export_path)
resave_tf_model(tf_export_path, final_tf_export_path)

print(f"Model successfully converted to TensorFlow SavedModel format and saved in {final_tf_export_path} directory")


Model successfully converted to TensorFlow SavedModel format and saved in /content/drive/MyDrive/airfoil_model_tf_final directory
